In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import Model
import os
checkpoint_path = "training_ResNet_50/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [2]:
#Importing the data from the directory
train_datagen= ImageDataGenerator()      
test_datagen = ImageDataGenerator()      
train_generator = train_datagen.flow_from_directory(
        'datasetjpgrgb/train',
        target_size=(150, 150),
        color_mode='rgb',
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        'datasetjpgrgb/val',
        target_size=(150, 150),
        color_mode='rgb',
        batch_size=32,
        class_mode='categorical')

Found 30000 images belonging to 3 classes.
Found 7500 images belonging to 3 classes.


In [3]:
preprocess_input=tf.keras.applications.resnet.preprocess_input     #Preprocessing the inputs for the ResNet

In [4]:
base_model = tf.keras.applications.resnet50.ResNet50(input_shape=(150,150,3),  #Loading The ResNet Model
                                               include_top=False,
                                               weights='imagenet')

In [5]:
base_model.trainable=False                                         #Keeping the training false 

In [6]:
base_model.summary()                                               #The Summary of the model

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 156, 156, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 75, 75, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 75, 75, 64)   256         ['conv1_conv[0][0]']      

In [7]:
last_layer = base_model.get_layer("conv5_block3_out")            #Getting the last layer of the ResNet
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output


last layer output shape:  (None, 5, 5, 2048)


In [8]:
#The input to the model
inputs=tf.keras.layers.Input(shape=(150,150,3))
a=preprocess_input(inputs)
last_output=base_model(a,training=False)

In [10]:
# This model ends with a convolutional layer of size(,5,5,2048)
# if directly feed to a dense layer of 1024 activations the number of parameters will be over 52 Million
# These parameters will cause slow training.
# To reduce this problem a pooling layer with size of 5X5 is used so as to reduce the dimension to 209K
y = tf.keras.layers.GlobalAveragePooling2D()(last_output)
# Flatten the output layer to 1 dimension
x = tf.keras.layers.Flatten()(y)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
# Add a dropOut Layer
x = tf.keras.layers.Dropout(0.02)(x)


In [11]:
# Add a final sigmoid layer for classification
prediction_layer = tf.keras.layers.Dense(3, activation='softmax')           
outputs=prediction_layer(x)

In [12]:
model=tf.keras.Model(inputs,outputs)

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07
),
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 150, 150, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 150, 150, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 5, 5, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0     

In [15]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq=2*256,
                                                 verbose=1)

model.fit(
        train_generator,
        steps_per_epoch=256,
        epochs=500,
        validation_data=validation_generator,
        validation_steps=100,
        callbacks=[cp_callback]
)

Epoch 1/500
 30/256 [==>...........................] - ETA: 5:43 - loss: 1.0487 - accuracy: 0.4280